In [ ]:
import yfinance as yf
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt

# Группы тикеров
groups = {
    "Equities": ["AAPL", "MSFT", "NVDA", "TSLA", "AMZN", "GOOG", "META", "JPM", "WMT", "NFLX", "BABA", "DIS", "PFE", "VZ", "KO", "INTC", "CSCO", "ADBE", "CMCSA", "T"],
    "Commodities": ["CL=F", "GC=F", "SI=F", "HG=F", "ZS=F"],
    "Equity Indices": ["^GSPC", "^DJI", "^IXIC", "^RUT", "^VIX"],
    "Bond Indices": ["^TNX", "^IRX", "^TYX"],
    "Currencies": ["EURUSD=X", "JPYUSD=X", "GBPUSD=X", "AUDUSD=X", "CADUSD=X", "CHFUSD=X", "CNYUSD=X", "SGDUSD=X", "HKDUSD=X"]
}

base_dict = {}

# Загрузка данных
for group, tickers in groups.items():
    for ticker in tqdm(tickers, desc=f"Загрузка данных для группы {group}"):
        base_dict[ticker] = yf.download(ticker, start='1990-01-01', interval='1d')

# Проверка структуры данных
def check_data_structure(ticker, df):
    print(f"\n--- Проверка структуры данных для {ticker} ---")

    # Проверка присутствия необходимых столбцов
    required_columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']
    for col in required_columns:
        if col not in df.columns:
            print(f"Столбец {col} отсутствует в данных {ticker}")
        else:
            print(f"Столбец {col} присутствует в данных {ticker}")

    # Проверка частоты данных
    print(f"\nЧастота данных для {ticker}:\n", pd.infer_freq(df.index))

    # Проверка на пропуски данных
    missing_values = df.isnull().sum()
    print(f"\nПропущенные значения в данных {ticker}:\n", missing_values)

    # Проверка на нерегулярные даты
    df['Date_Difference'] = df.index.to_series().diff().dt.days
    irregular_dates = df[df['Date_Difference'] > 1]
    if not irregular_dates.empty:
        print(f"\nНайдены нерегулярные даты для {ticker}:\n", irregular_dates[['Date_Difference']])
    else:
        print(f"\nНерегулярных дат не найдено для {ticker}.")

# Основной цикл для проведения анализа структуры данных для каждой группы
for group, tickers in groups.items():
    print(f"\n=== Анализ структуры данных для группы {group} ===")
    for ticker in tickers:
        df = base_dict[ticker]
        check_data_structure(ticker, df)


In [ ]:
# Ресемплирование и вывод результатов
def resample_data(df, freq, group_name, ticker_name):
    df_resampled = df['Close'].resample(freq).mean()

    # Интерполяция
    df_resampled = df_resampled.interpolate(method='linear')

    # Проверка на достаточное количество наблюдений для STL-декомпозиции
    if len(df_resampled) < 24:
        print(f"Недостаточно данных для декомпозиции для {ticker_name} с частотой {freq}. Пропуск...")
        return

    # Скользящее среднее
    rolling_mean = df_resampled.rolling(window=50).mean()

    # STL декомпозиция
    result_stl = seasonal_decompose(df_resampled, model='additive', period=12)

    # Z-score для выбросов
    z_score = (df_resampled - df_resampled.mean()) / df_resampled.std()
    outliers = z_score[np.abs(z_score) > 3]

    # Визуализация
    plt.figure(figsize=(14, 10))

    plt.subplot(4, 1, 1)
    plt.plot(df_resampled, label=f'Resampled ({freq}) Close Prices')
    plt.plot(rolling_mean, label=f'Rolling Mean', color='orange')
    plt.title(f'{ticker_name} ({group_name}) - {freq} Resample and Rolling Mean')
    plt.legend()

    plt.subplot(4, 1, 2)
    plt.plot(result_stl.trend, label='Trend', color='green')
    plt.title(f'{ticker_name} ({group_name}) - STL Trend Component')
    plt.legend()

    plt.subplot(4, 1, 3)
    plt.plot(result_stl.seasonal, label='Seasonality', color='purple')
    plt.title(f'{ticker_name} ({group_name}) - STL Seasonal Component')
    plt.legend()

    plt.subplot(4, 1, 4)
    plt.scatter(outliers.index, outliers, color='red', label='Outliers (Z-score > 3)')
    plt.title(f'{ticker_name} ({group_name}) - Detected Outliers')
    plt.legend()

    plt.tight_layout()
    plt.show()

# Функция для проведения анализа с разными частотами ресемплирования
def compare_resampling(df, group_name, ticker_name):
    freqs = ['D', 'W', 'M', 'Q', 'A']  # День, Неделя, Месяц, Квартал, Год

    for freq in freqs:
        print(f"Ресемплирование с частотой: {freq}")
        resample_data(df, freq, group_name, ticker_name)

# Анализ данных для всех групп с разными частотами ресемплирования
for group_name, tickers in groups.items():
    for ticker in tickers:
        df = base_dict[ticker]
        df.index = pd.to_datetime(df.index)
        df = df.dropna()  # Удаление пропусков для чистоты эксперимента
        compare_resampling(df, group_name, ticker)


In [ ]:
import seaborn as sns
from datetime import datetime



# Описательная статистика и агрегирование по группе
def calculate_statistics_and_volatility(base_dict):
    for group_name, tickers in groups.items():
        print(f"\n=== Описательная статистика для группы {group_name} ===")
        group_stats = []

        for ticker in tickers:
            df = base_dict[ticker]

            # Опис статистика
            descriptive_stats = df['Close'].describe()
            variance = df['Close'].var()
            median = df['Close'].median()

            print(f"\nСтатистические характеристики для {ticker}:")
            print(descriptive_stats)
            print(f"\nДисперсия для {ticker}: {variance}")
            print(f"Медиана для {ticker}: {median}")

            group_stats.append(df['Close'].describe())

        # Агрегированная статистика для группп
        aggregated_stats = pd.DataFrame(group_stats).mean()
        print(f"\n=== Агрегированная описательная статистика для группы {group_name} ===")
        print(aggregated_stats)


calculate_statistics_and_volatility(base_dict)

# Функция для расчета волатильности на основе скользящего окна
def calculate_moving_volatility(df, window=30):
    """
    Функция для расчета волатильности на основе скользящего окна.
    :param df: DataFrame с данными активов
    :param window: Размер скользящего окна в днях
    :return: Series с рассчитанной волатильностью
    """
    returns = df['Close'].pct_change()  # Процентные изменения
    volatility = returns.rolling(window=window).std() * np.sqrt(252)  # Годовая волатильность
    return volatility

# Функция для расчета волатильности для всей группы и построения графиков
def calculate_all_group_volatility(window=30):
    for group_name, tickers in groups.items():
        group_volatilities = []
        print(f"\n=== Волатильность для группы {group_name} ===")

        # Рассчитываем волатильность для каждого актива в группе
        for ticker in tickers:
            df = base_dict[ticker]
            volatility = calculate_moving_volatility(df, window)

            # Построение графика волатильности для каждого актива
            plt.figure(figsize=(10, 6))
            plt.plot(df.index, volatility, label=f'{window}-Day Moving Volatility for {ticker}')
            plt.title(f'Изменение волатильности за весь период для {ticker}')
            plt.xlabel('Дата')
            plt.ylabel('Волатильность')
            plt.legend()
            plt.show()

            group_volatilities.append(volatility)

        # Агрегированная волатильность
        group_volatility_avg = pd.concat(group_volatilities, axis=1).mean(axis=1)

        plt.figure(figsize=(10, 6))
        plt.plot(group_volatility_avg.index, group_volatility_avg, label=f'{window}-Day Avg Volatility for {group_name}')
        plt.title(f'Агрегированная волатильность для группы {group_name}')
        plt.xlabel('Дата')
        plt.ylabel('Агрегированная волатильность')
        plt.legend()
        plt.show()


calculate_all_group_volatility(window=30)

# Расчет корреляций и heatmap
def calculate_group_correlations(base_dict):
    group_close_prices = {}

    for group, tickers in groups.items():
        group_data = []
        for ticker in tickers:
            df = base_dict[ticker]
            if 'Close' in df.columns:
                group_data.append(df['Close'])
        df_group = pd.concat(group_data, axis=1)
        df_group.columns = tickers
        group_close_prices[group] = df_group

    # Все возможные пары групп
    group_names = list(group_close_prices.keys())
    group_pairs = [(group1, group2) for i, group1 in enumerate(group_names) for group2 in group_names[i+1:]]

    # Цикл по парам групп для построения тепловых карт корреляций
    for group1, group2 in group_pairs:
        print(f"\nКорреляция между группами {group1} и {group2}:")

        # Данные для обеих групп
        df_group1 = group_close_prices[group1].pct_change()
        df_group2 = group_close_prices[group2].pct_change()

        # Объединяем данные по обеим группам
        combined_df = pd.concat([df_group1, df_group2], axis=1)

        # Рассчитываем корреляцию между всеми активами из двух групп
        correlation_matrix = combined_df.corr()

        # Увеличиваем размер графика и убираем аннотации
        plt.figure(figsize=(16, 12))
        sns.heatmap(correlation_matrix, annot=False, cmap='coolwarm', linewidths=0.5)
        plt.title(f'Корреляция между {group1} и {group2}')
        plt.xticks(rotation=45, ha='right')
        plt.show()

calculate_group_correlations(base_dict)


In [ ]:
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import matplotlib.pyplot as plt

# Функция для выполнения ADF теста (теста Дикки-Фуллера)
def adf_test(df, title=""):
    print(f"\n{title}")
    result = adfuller(df.dropna())
    print(f'ADF Statistic: {result[0]}')
    print(f'p-value: {result[1]}')
    print('Critical Values:')
    for key, value in result[4].items():
        print(f'   {key}: {value}')
    if result[1] < 0.05:
        print("Ряд стационарен (отклоняем нулевую гипотезу)")
    else:
        print("Ряд нестационарен (не отклоняем нулевую гипотезу)")

# Построение ACF и PACF
def plot_acf_pacf(df, title="", lags=40):
    # ACF (Автокорреляционная функция)
    plt.figure(figsize=(12, 6))
    plot_acf(df.dropna(), lags=lags)
    plt.title(f'ACF {title}')
    plt.show()

    # PACF (Частичная автокорреляционная функция)
    plt.figure(figsize=(12, 6))
    plot_pacf(df.dropna(), lags=lags)
    plt.title(f'PACF {title}')
    plt.show()

# Функция для выполнения тестов для каждого актива в группе
def perform_individual_ts_tests(group_name, tickers, lags=40):
    for ticker in tickers:
        df = base_dict[ticker]
        print(f"\nТестирование для актива {ticker} из группы {group_name}:")

        # Выполняем тест Дикки-Фуллера (ADF)
        adf_test(df['Close'], title=f"ADF для {ticker}")

        # Построение графиков ACF и PACF
        plot_acf_pacf(df['Close'], title=f"для {ticker}", lags=lags)

# Функция для вычисления среднего значения Close для группы активов
def get_group_average(df_group):
    return df_group.mean(axis=1)

# Тесты на стационарность и автокорреляцию для группы активов
def perform_group_ts_tests(group_name, tickers, lags=40):
    # Собираем данные для группы
    group_data = []
    for ticker in tickers:
        df = base_dict[ticker]
        group_data.append(df['Close'])

    # Рассчитываем среднее по группе
    df_group = pd.concat(group_data, axis=1)
    group_avg = get_group_average(df_group)

    # Выполняем тесты для среднего по группе
    print(f"\nТестирование для группы {group_name}:")

    # Выполняем тест Дикки-Фуллера (ADF) для среднего по группе
    adf_test(group_avg, title=f"ADF для группы {group_name}")

    # Построение ACF и PACF для среднего по группе
    plot_acf_pacf(group_avg, title=f"для группы {group_name}", lags=lags)

# Основная функция для выполнения тестов на стационарность и автокорреляцию как для индивидуальных активов, так и для групп
def perform_ts_tests_for_all_groups():
    for group_name, tickers in groups.items():
        # Тесты для каждого актива в группе
        perform_individual_ts_tests(group_name, tickers, lags=40)

        # Тесты для группы в целом (агрегированные данные)
        perform_group_ts_tests(group_name, tickers, lags=40)

perform_ts_tests_for_all_groups()
